In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import gqcpy
import numpy as np
import psi4

In [2]:
psi4.set_memory("4 GB")

4000000000

- Work in a spin-orbital basis: $\alpha$ and $\beta$ have separate Fukui matrices.
-

# Preliminary functions

In [3]:
def dimensions(molecule):
    """
    Return the dimensions (N, M) of a molecule in a given basis. Here, N is the number of electrons and M the number of spinors.
    """
    N = molecule.numberOfElectrons()

    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    M = spin_orbital_basis.numberOfSpinors()
    
    return (N, M)

In [4]:
def onv_bases(N, K):
    """
    Create three ONV bases: I for the (N-1) electron system, J for the N-electron system and K for the (N+1)-electron system.
    """

    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2
    
    onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta - 1)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
    onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
    onv_basis_K = gqcpy.SpinResolvedONVBasis(K, N_alpha + 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

    return (onv_basis_I, onv_basis_J, onv_basis_K)

In [5]:
def C_RHF(molecule):
    """
    Generate restricted spinors, defined as the expansion coefficients in the scalar AO basis, from a restricted
    Hartree-Fock calculation.
    """
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    #print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [6]:
def C_UHF(molecule):
    """
    Generate unrestricted spin-orbitals, for each spin component defined as the expansion coefficients 
    in the scalar AO basis, from an unrestricted Hartree-Fock calculation.
    """
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    #print("N_alpha: {}\nN_beta: {}\nK: {}".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    return uhf_parameters.expansion()

In [7]:
def optimize_geometry(molecule):
    """
    Get optimized geometry of a given molecule using a B3LYP calculation with a 6-311++G(2d,2p) basis set. 
    This method uses Psi4 for its calculations.
    """
    psi4.set_options({'reference': 'UKS',
                     'basis': '6-311++G(2d,2p)'})

    psi4.optimize('scf', molecule = molecule, dft_functional='b3lyp')
    
    return molecule

In [49]:
def TM(C_ion, C_neutral):
    """
    Return a transformation matrix that expresses a neutral spinor/spin-orbital in terms of the ones of the ionized molecule.
    """
    return np.linalg.inv(C_ion) @ C_neutral

In [109]:
def ROHF(molecule):
    
    psi4.set_options({'reference': 'rohf',
                'basis': '6-311++G(2d,2p)',
                's_orthogonalization': 'canonical',
                 'scf_type': 'df'})
    
    # get wavefunction for the neutral compound
    E_J, Psi_J = psi4.energy('scf', molecule = ch2nh, return_wfn = True)
    
    # get wavefunction for the positive ion
    ch2nh.set_molecular_charge(+1)
    ch2nh.set_multiplicity(2)
    E_I, Psi_I = psi4.energy('scf', molecule = ch2nh, return_wfn = True)
    
    # Ca = Psi_J.Ca().to_array() # !!! uitzoeken of elektron altijd bij beta weggenomen wordt !!!
    Cb_J = Psi_J.Cb().to_array()
    Cb_I = Psi_I.Cb().to_array()
    
    #Da = Ca.T @ Psi_J.Da().to_array() @ Ca.conj() # alpha density matrix in terms of MOs
    Db_J = np.linalg.inv(Cb_J.conj()) @ Psi_J.Db().to_array() @ np.linalg.inv(Cb_J.T) # beta density matrix of Psi_J in terms of its MOs
    
    Db_I = np.linalg.inv(Cb_I.conj()) @ Psi_I.Db().to_array() @ np.linalg.inv(Cb_I.T) # beta density matrix of Psi_I in terms of its MOs
    
    Ub_IJ = TM(Cb_I, Cb_J)
    
    Fb = Db_J - Ub_IJ.T @ Db_I @ Ub_IJ.conj()
    
    F_eigenvalues, F_naturals = np.linalg.eigh(Fb)
    
    return (F_eigenvalues, F_naturals)

$$\require{mhchem}$$       

## $\ce{CH_2NH}$

In [74]:
name = "ch2h"

ch2nh = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

C 0.0580 0.5852 0.0000
N 0.0580 -0.6875 0.0000
H -0.8313 1.2172 0.0000
H 1.0094 1.1151 0.0000
H -0.9323 -1.0310 0.0000
"""
)

In [75]:
optimize_geometry(ch2nh)

Optimizer: Optimization complete!


In [76]:
psi4.set_options({'reference': 'rohf',
                'basis': '6-311++G(2d,2p)',
                's_orthogonalization': 'canonical',
                 'scf_type': 'df'})

energy, wfn = psi4.energy('scf', molecule = ch2nh, return_wfn = True)

In [77]:
Ca = wfn.Ca().to_array()
Cb = wfn.Cb().to_array()

In [78]:
Da = wfn.Da().to_array()
Db = wfn.Db().to_array()

In [79]:
Da_MO = np.linalg.inv(Ca.conj()) @ Da @ np.linalg.inv(Ca.T)
Db_MO = np.linalg.inv(Cb.conj()) @ Da @ np.linalg.inv(Cb.T)

In [80]:
ch2nh.set_molecular_charge(+1)
ch2nh.set_multiplicity(2)

In [81]:
energy_pos, wfn_pos = psi4.energy('scf', molecule = ch2nh, return_wfn = True)



felez: [array([6., 7., 1., 1., 1.])]
all_fc_known: True
all_fm_known: True
zel: 16.0
fzel: [16.0]
c: [1.0, 0]
fc: [0.0]
m: [2]
fm: [1]
Assess candidate (1.0, (0.0,), 2, (1,)): 1   2   3   4   4-0 5   5-0 6   6-0 7   7-0
Assess candidate (1.0, (0.0,), 2, (1,)): T       T   T   T   T   T   T   T   T   T   --> False
Assess candidate (0, (0.0,), 2, (1,)): T   T   T   T   T       T       T   T   T   --> False
Following warning is harmless if you've altered chgmult through `set_molecular_change` or `set_multiplicity`. Such alterations are an expert feature. Specifying in the original molecule string is preferred.


felez: [array([6., 7., 1., 1., 1.])]
all_fc_known: True
all_fm_known: True
zel: 16.0
fzel: [16.0]
c: [1.0, 0]
fc: [0.0]
m: [2]
fm: [1]
Assess candidate (1.0, (0.0,), 2, (1,)): 1   2   3   4   4-0 5   5-0 6   6-0 7   7-0
Assess candidate (1.0, (0.0,), 2, (1,)): T       T   T   T   T   T   T   T   T   T   --> False
Assess candidate (0, (0.0,), 2, (1,)): T   T   T   T   T       T  

In [82]:
Ca_pos = wfn_pos.Ca().to_array()
Cb_pos = wfn_pos.Cb().to_array()

In [83]:
Da_pos = wfn_pos.Da().to_array()
Db_pos = wfn_pos.Db().to_array()

In [84]:
Da_pos_MO = np.linalg.inv(Ca_pos.conj()) @ Da_pos @ np.linalg.inv(Ca_pos.T)
Db_pos_MO = np.linalg.inv(Cb_pos.conj()) @ Db_pos @ np.linalg.inv(Cb_pos.T)

In [85]:
Ua = np.linalg.inv(Ca_pos) @ Ca
Ub = np.linalg.inv(Cb_pos) @ Cb

In [86]:
Fa = Da_MO - np.linalg.inv(Ua.conj()) @ Da_pos_MO @ np.linalg.inv(Ua.T)
Fb = Db_MO - np.linalg.inv(Ub.conj()) @ Db_pos_MO @ np.linalg.inv(Ub.T)

In [87]:
eigenvalues, eigenvectors = np.linalg.eigh(Fb)

In [88]:
eigenvalues.max()

0.9999999999999997

In [89]:
eigenvalues.min()

-0.16094756691081244

$$\require{mhchem}$$       

## $\ce{CH_2NH}$

In [124]:
name = "ch2h"

ch2nh = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

C 0.0580 0.5852 0.0000
N 0.0580 -0.6875 0.0000
H -0.8313 1.2172 0.0000
H 1.0094 1.1151 0.0000
H -0.9323 -1.0310 0.0000
"""
)

In [125]:
optimize_geometry(ch2nh)

Optimizer: Optimization complete!


In [126]:
%%capture
# to suppress the output of Psi4

F_eigenvalues, _ = ROHF(ch2nh)

In [127]:
F_eigenvalues.max()

0.9999999999999992

In [128]:
F_eigenvalues.min()

-0.16094756691081144